In [8]:
import pandas as pd
import os 
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import math
from tqdm import tqdm
tqdm.pandas()
import glob

In [2]:
train = gpd.read_file('data/train/train.shp')
train.dropna(subset=['geometry'], inplace = True)
train.reset_index(drop=True, inplace= True)
test = gpd.read_file('data/test/test.shp')
print(train.info())
print(test.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 5 columns):
Field_Id      2494 non-null int64
Area          2494 non-null float64
Subregion     2494 non-null int64
Crop_Id_Ne    2494 non-null object
geometry      2494 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 97.5+ KB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 4 columns):
Field_Id     1074 non-null int64
Area         1074 non-null float64
Subregion    1074 non-null int64
geometry     1074 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 33.6+ KB
None


In [16]:
train.head()

,Field_Id,Area,Subregion,Crop_Id_Ne,geometry
0,1,0.915905,3,8,POLYGON ((2467881.175041331 -3352032.059296422...
1,2,2.064410,3,6,"POLYGON ((2468428.90623293 -3351993.17305059, ..."
2,3,1.080300,3,8,POLYGON ((2468333.745293362 -3352058.485056543...
3,4,1.316190,3,8,POLYGON ((2468607.857733211 -3352012.337531495...
4,7,5.529220,3,8,"POLYGON ((2468646.08520868 -3351706.201920062,..."


In [21]:
train_dates_files = sorted(glob.glob("stats/train/*.csv"))
print(train_dates_files)
print(len(train_dates_files))

['stats/train/train_neighbors5.csv', 'stats/train/train_stats2017-01-01.csv', 'stats/train/train_stats2017-01-31.csv', 'stats/train/train_stats2017-02-10.csv', 'stats/train/train_stats2017-03-12.csv', 'stats/train/train_stats2017-03-22.csv', 'stats/train/train_stats2017-05-31.csv', 'stats/train/train_stats2017-06-20.csv', 'stats/train/train_stats2017-07-10.csv', 'stats/train/train_stats2017-07-15.csv', 'stats/train/train_stats2017-08-04.csv', 'stats/train/train_stats2017-08-19.csv']
12


In [26]:
train_final = train.copy()
train_final.drop(['geometry'], axis=1, inplace = True)
train_final.reset_index(drop= True, inplace = True)
# print(train_final.head(2))
for df_path in train_dates_files:
    df = pd.read_csv(df_path)
    if(not df['Field_Id'].equals(train_final['Field_Id'])):
        print(False)
    df = df.drop(['Field_Id'], axis=1)
    df.reset_index(drop=True, inplace=True) # I have to reset index
    train_final.reset_index(drop=True, inplace=True)
#     train_final = pd.concat([train_final,df], axis=1, ignore_index=True)
    train_final = pd.concat([train_final,df], axis=1)
print(train_final.head(2))

   Field_Id      Area  Subregion Crop_Id_Ne  occ_neigh_1  occ_neigh_2  \
0         1  0.915905          3          8            0            0   
1         2  2.064410          3          6            0            0   

   occ_neigh_3  occ_neigh_4  occ_neigh_5  occ_neigh_6          ...           \
0            0            0            0            2          ...            
1            0            1            0            0          ...            

   ndvi_val112017-08-19  ndvi_val122017-08-19  ndvi_val132017-08-19  \
0              0.237607              0.234053              0.241777   
1              0.340261              0.429630              0.472712   

   ndvi_val142017-08-19  ndvi_val152017-08-19  ndvi_val162017-08-19  \
0              0.265228              0.283237              0.284668   
1              0.458003              0.436379              0.429870   

   ndvi_val172017-08-19  ndvi_val182017-08-19  ndvi_val192017-08-19  \
0              0.283370              0.2898

In [27]:
test_dates_files = sorted(glob.glob("stats/test/*.csv"))
print(test_dates_files)

test_final = test.copy()
test_final.drop(['geometry'], axis=1, inplace = True)
test_final.reset_index(drop= True, inplace = True)

for df_path in test_dates_files:
    df = pd.read_csv(df_path)
    if(not df['Field_Id'].equals(test_final['Field_Id'])):
        print(False)
    df = df.drop(['Field_Id'], axis=1)
    df.reset_index(drop=True, inplace=True) # I have to reset index
    test_final.reset_index(drop=True, inplace=True)
    test_final = pd.concat([test_final,df], axis=1)
                           
test_final.head()

['stats/test/test_neighbors5.csv', 'stats/test/test_stats2017-01-01.csv', 'stats/test/test_stats2017-01-31.csv', 'stats/test/test_stats2017-02-10.csv', 'stats/test/test_stats2017-03-12.csv', 'stats/test/test_stats2017-03-22.csv', 'stats/test/test_stats2017-05-31.csv', 'stats/test/test_stats2017-06-20.csv', 'stats/test/test_stats2017-07-10.csv', 'stats/test/test_stats2017-07-15.csv', 'stats/test/test_stats2017-08-04.csv', 'stats/test/test_stats2017-08-19.csv']


,Field_Id,Area,Subregion,occ_neigh_1,occ_neigh_2,occ_neigh_3,occ_neigh_4,occ_neigh_5,occ_neigh_6,occ_neigh_7,...,ndvi_val112017-08-19,ndvi_val122017-08-19,ndvi_val132017-08-19,ndvi_val142017-08-19,ndvi_val152017-08-19,ndvi_val162017-08-19,ndvi_val172017-08-19,ndvi_val182017-08-19,ndvi_val192017-08-19,ndvi_val202017-08-19
0,5,1.322590,3,0,0,0,1,0,1,0,...,0.238046,0.254771,0.259094,0.259139,0.275862,0.258755,0.244924,0.257009,0.270404,0.264289
1,6,0.955864,3,0,0,0,1,0,1,0,...,0.269110,0.250118,0.271930,0.276095,0.259727,0.236168,0.241813,0.270466,0.252029,0.266129
2,10,11.509800,3,0,0,0,0,0,0,0,...,0.254404,0.260175,0.241796,0.275171,0.281527,0.262645,0.274573,0.297929,0.335245,0.399798
3,18,2.697520,3,0,0,0,0,0,0,0,...,0.256779,0.263705,0.261575,0.253454,0.233981,0.222408,0.210818,0.248604,0.291473,0.298613
4,23,4.238030,3,0,0,1,0,1,0,1,...,0.184266,0.171910,0.231088,0.269857,0.258275,0.221536,0.234970,0.235560,0.235999,0.241856


In [29]:
train_final.to_csv('stats/train_final.csv', index = False)
test_final.to_csv('stats/test_final.csv', index = False)